# Importing data from the Danish building registry (BBR)

This document shows how to retrieve data from the Danish building registry, BBR, and insert it into a relational database for easier use. <br>

As prerequisites, you need to be able to run Python and have a PostgresQL database ready to receive the BBR data.

## Obtaining the data

1. Go to https://datafordeler.dk/ <br>
Click on "log in" and create a new web user. You should get a user name and password to access datafordeler. <br>
<br>
2. With your username and password, log in to the datafordeler self-service to retrieve data: <br> https://selfservice.datafordeler.dk/ <br>
<br>
3. It's not intuitive, but your account is linked to several "users", each with different permissions. Check the Users tab (Brugere) - if you only have the user "Webbruger", you need to create a new one. Click on the + tab and create a service user with the "user name and access code" method.<br>
<br>
4. You are now ready to request public data on datafordeler. Go to the Downloads tab (Filudtræk). You should see an empty field - that's because you haven't requested data yet. To get access to data, you need to create a download. You have three choices:<br> - Clicking Opret will allow you to create a permanent download button, that is kept up to date and that you can use multiple times.<br> - Clicking Download will allow you to request a one-time download of the dataset.<br> - Clicking Predefined will allow you to download a dataset with a fixed set of parameters (instead of customizing everything). In particular, you can use Predefined to download the BBR dataset with only up-to-date entries, in JSON or XML format. <br>
<br>
5. You should now see a list of all available downloads. Give your download a name (Visningsnavn) and select BBR Totaludtræk in the list (or BBR Aktuelt Totaludtræk if using Predefined). Click Next. If you chose Opret or Download, you can now adjust a lot of parameters, such as downloading entries for only a specific municipality. If you used Predefined, the parameters are locked.<br><br>

6. Click Save (Gem). You will be taken back to the Download tabs. If you used Opret or Predefined, you should see your data subscription there. You can modify or delete it if you don't think that you will need to download it again in the future. You will receive an email with information on how to get your data.<br>
<br>
7. Actually getting the data is a bit tricky: you cannot download it from Datafordeler directly. You need to use a FTP client like https://filezilla-project.org/ <br>
Download and install FileZilla. When you launch it, enter the address provided in the email you got from Datafordeler, as well as your Datafordeler service user number (*not* your initial username: this is the user number you created in step 3) and password. Click Connect, and you should finally be able to see and download your files! <br>

## Setup

In [1]:
import psycopg as pg # Package to communicate between Python and PostgresQL
import ijson
from IPython.display import clear_output
params='dbname=BBR user=postgres password=mypassword' # Write the parameters to connect to your PostgresQL database here

In [2]:
jsonfile='filepath.json' # Write the filepath to your json file containing BBR data

It's useful to define a simple function to run SQL code in Python using the psycopg package. This way we can run any SQL operation directly from Python.

In [3]:
def run_sql (DBparameters,SQLcode): # Takes a string of PostgresQL code and runs it on the PostgresQL database
    try:
        # connect to the PostgresQL database
        connector = pg.connect(DBparameters)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLcode)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)

    finally:
        if connector is not None:
            connector.close()

### Creating PostgresQL tables corresponding to all BBR items and parameters

There are a lot of items in the BBR database, each with many parameters! You'll probably want to filter which ones are important in your project, but the code below will support importing the entire database, with all parameters and items. If you have enough storage space and processing power, importing everything allows you to make sure you're not missing a parameter you might need later. I gave each parameter a type that should fit the BBR data, but feel free to change that if you encounter any error (for instance change the value of "character varying" if you encounter any entry that's longer than the limit I defined). Finally, keep in mind that the structure of BBR might have changed after this code was written, and I'm unlikely to keep it up to date!

In [12]:
SQLcreateall="""
BEGIN;

CREATE TABLE IF NOT EXISTS public.bygninglist
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    byg007Bygningsnummer integer,
    byg021BygningensAnvendelse character varying(50) COLLATE pg_catalog."default",
    byg024AntalLejlighederMedKøkken smallint, 
    byg025AntalLejlighederUdenKøkken smallint,
    byg026Opførelsesår smallint,
    byg027OmTilbygningsår smallint,
    byg029DatoForMidlertidigOpførtBygning character varying(50) COLLATE pg_catalog."default",
    byg030Vandforsyning character varying(50) COLLATE pg_catalog."default" ,
    byg031Afløbsforhold character varying(50) COLLATE pg_catalog."default" ,
    byg032YdervæggensMateriale smallint,
    byg033Tagdækningsmateriale smallint,
    byg034SupplerendeYdervæggensMateriale smallint,
    byg035SupplerendeTagdækningsMateriale smallint,
    byg036AsbestholdigtMateriale smallint,
    byg037KildeTilBygningensMaterialer character varying(50) COLLATE pg_catalog."default" ,
    byg038SamletBygningsareal integer,
    byg039BygningensSamledeBoligAreal integer,
    byg040BygningensSamledeErhvervsAreal integer,
    byg041BebyggetAreal integer,
    byg042ArealIndbyggetGarage smallint,
    byg043ArealIndbyggetCarport smallint,
    byg044ArealIndbyggetUdhus smallint,
    byg045ArealIndbyggetUdestueEllerLign smallint,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen integer,
    byg047ArealAfAffaldsrumITerrænniveau integer,
    byg048AndetAreal integer,
    byg049ArealAfOverdækketAreal integer,
    byg050ArealÅbneOverdækningerPåBygningenSamlet integer,
    byg051Adgangsareal integer,
    byg052BeregningsprincipCarportAreal character varying(50) COLLATE pg_catalog."default",
    byg053BygningsarealerKilde character varying(50) COLLATE pg_catalog."default",
    byg054AntalEtager smallint,
    byg055AfvigendeEtager smallint,
    byg056Varmeinstallation smallint,
    byg057Opvarmningsmiddel smallint,
    byg058SupplerendeVarme smallint,
    byg069Sikringsrumpladser character varying(50) COLLATE pg_catalog."default",
    byg070Fredning character varying(50) COLLATE pg_catalog."default",
    byg071BevaringsværdighedReference character varying(50) COLLATE pg_catalog."default",
    byg094Revisionsdato character varying(50) COLLATE pg_catalog."default",
    byg111StormrådetsOversvømmelsesSelvrisiko character varying(50) COLLATE pg_catalog."default",
    byg112DatoForRegistreringFraStormrådet character varying(50) COLLATE pg_catalog."default",
    byg113Byggeskadeforsikringsselskab character varying(50) COLLATE pg_catalog."default",
    byg114DatoForByggeskadeforsikring character varying(50) COLLATE pg_catalog."default",
    byg119Udledningstilladelse character varying(50) COLLATE pg_catalog."default",
    byg121OmfattetAfByggeskadeforsikring character varying(50) COLLATE pg_catalog."default",
    byg122Gyldighedsdato character varying(50) COLLATE pg_catalog."default",
    byg123MedlemskabAfSpildevandsforsyning character varying(50) COLLATE pg_catalog."default",
    byg124PåbudVedrSpildevandsafledning character varying(50) COLLATE pg_catalog."default",
    byg125FristVedrSpildevandsafledning character varying(50) COLLATE pg_catalog."default",
    byg126TilladelseTilUdtræden character varying(50) COLLATE pg_catalog."default",
    byg127DatoForTilladelseTilUdtræden character varying(50) COLLATE pg_catalog."default",
    byg128TilladelseTilAlternativBortskaffelseEllerAfledning character varying(50) COLLATE pg_catalog."default",
    byg129DatoForTilladelseTilAlternativBortskaffelseEllerAfledning character varying(50) COLLATE pg_catalog."default",
    byg130ArealAfUdvendigEfterisolering integer,
    byg131DispensationFritagelseIftKollektivVarmeforsyning character varying(50) COLLATE pg_catalog."default",
    byg132DatoForDispensationFritagelseIftKollektivVarmeforsyning character varying(50) COLLATE pg_catalog."default",
    byg133KildeTilKoordinatsæt character varying(50) COLLATE pg_catalog."default",
    byg134KvalitetAfKoordinatsæt character varying(50) COLLATE pg_catalog."default",
    byg135SupplerendeOplysningOmKoordinatsæt character varying(50) COLLATE pg_catalog."default",
    byg136PlaceringPåSøterritorie character varying(50) COLLATE pg_catalog."default",
    byg137BanedanmarkBygværksnummer character varying(50) COLLATE pg_catalog."default",
    byg301TypeAfFlytning character varying(50) COLLATE pg_catalog."default",
    byg302Tilflytterkommune character varying(50) COLLATE pg_catalog."default",
    byg403ØvrigeBemærkningerFraStormrådet character varying(50) COLLATE pg_catalog."default",
    byg404Koordinat character varying(50) COLLATE pg_catalog."default",
    byg406Koordinatsystem character varying(50) COLLATE pg_catalog."default",
    byg500Notatlinjer character varying(50) COLLATE pg_catalog."default",
    byg150Gulvbelægning character varying(50) COLLATE pg_catalog."default",
    byg151Frihøjde character varying(50) COLLATE pg_catalog."default",
    byg152ÅbenLukketKonstruktion character varying(50) COLLATE pg_catalog."default",
    byg153Konstruktionsforhold character varying(50) COLLATE pg_catalog."default",
    byg140ServitutForUdlejningsEjendomDato character varying(50) COLLATE pg_catalog."default",
    jordstykke integer,
    husnummer character varying(50) COLLATE pg_catalog."default",
    ejerlejlighed character varying(50) COLLATE pg_catalog."default",
    grund character varying(50) COLLATE pg_catalog."default",
    CONSTRAINT bygninglist_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.bbrsaglist
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    sag001Byggesagsnummer character varying(50) COLLATE pg_catalog."default" ,
    sag002Byggesagsdato character varying(50) COLLATE pg_catalog."default" ,
    sag003Byggetilladelsesdato character varying(50) COLLATE pg_catalog."default" ,
    sag004ForventetPåbegyndelsesdato character varying(50) COLLATE pg_catalog."default" ,
    sag005Påbegyndelsesdato character varying(50) COLLATE pg_catalog."default" ,
    sag006IbrugtagningsTilladelse character varying(50) COLLATE pg_catalog."default" ,
    sag007Henlæggelse character varying(50) COLLATE pg_catalog."default" ,
    sag008FærdigtBygningsareal character varying(50) COLLATE pg_catalog."default" ,
    sag009ForventetFuldførtDato character varying(50) COLLATE pg_catalog."default" ,
    sag010FuldførelseAfByggeri character varying(50) COLLATE pg_catalog."default" ,
    sag012Byggesagskode character varying(50) COLLATE pg_catalog."default" ,
    sag013AnmeldelseAfByggearbejde character varying(50) COLLATE pg_catalog."default" ,
    sag016DelvisIbrugtagningsTilladelse character varying(50) COLLATE pg_catalog."default" ,
    sag017ForeløbigFærdiggjortBygningsareal character varying(50) COLLATE pg_catalog."default" ,
    sag018ForeløbigFærdiggjortAntalLejligheder character varying(50) COLLATE pg_catalog."default" ,
    sag019Bygherreforhold character varying(50) COLLATE pg_catalog."default" ,
    sag024DatoForModtagelseAfAnsøgningOmByggetilladelse character varying(50) COLLATE pg_catalog."default" ,
    sag025DatoForFyldestgørendeAnsøgning character varying(50) COLLATE pg_catalog."default" ,
    sag026DatoForNaboorientering character varying(50) COLLATE pg_catalog."default" ,
    sag027DatoForFærdigbehandlingAfNaboorientering character varying(50) COLLATE pg_catalog."default" ,
    sag032Litra character varying(50) COLLATE pg_catalog."default" ,
    sag033ForeløbigFærdiggjortAntalLejlighederUdenKøkken character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT bbrsaglist_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.BygningEjendomsrelationList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    bygningPåFremmedGrund character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT BygningEjendomsrelationList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.EjendomsrelationList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    bfeNummer character varying(20) COLLATE pg_catalog."default" ,
    ejendommensEjerforholdskode character varying(50) COLLATE pg_catalog."default" ,
    ejendomsnummer character varying(50) COLLATE pg_catalog."default" ,
    vurderingsejendomsnummer character varying(50) COLLATE pg_catalog."default" ,
    ejerlejlighedsnummer character varying(50) COLLATE pg_catalog."default" ,
    ejendomstype character varying(50) COLLATE pg_catalog."default" ,
    tinglystAreal character varying(50) COLLATE pg_catalog."default" ,
    bygningPåFremmedGrund character varying(50) COLLATE pg_catalog."default" ,
    ejerlejlighed character varying(50) COLLATE pg_catalog."default" ,
    samletFastEjendom character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT EjendomsrelationList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.EnhedList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    enh008UUIDTilModerlejlighed character varying(50) COLLATE pg_catalog."default" ,
    enh020EnhedensAnvendelse character varying(50) COLLATE pg_catalog."default" ,
    enh023Boligtype character varying(50) COLLATE pg_catalog."default" ,
    enh024KondemneretBoligenhed character varying(50) COLLATE pg_catalog."default" ,
    enh025OprettelsesdatoForEnhedensIdentifikation character varying(50) COLLATE pg_catalog."default" ,
    enh026EnhedensSamledeAreal integer,
    enh027ArealTilBeboelse integer,
    enh028ArealTilErhverv integer,
    enh030KildeTilEnhedensArealer character varying(50) COLLATE pg_catalog."default" ,
    enh031AntalVærelser integer,
    enh032Toiletforhold character varying(50) COLLATE pg_catalog."default" ,
    enh033Badeforhold character varying(50) COLLATE pg_catalog."default" ,
    enh034Køkkenforhold character varying(50) COLLATE pg_catalog."default" ,
    enh035Energiforsyning character varying(50) COLLATE pg_catalog."default" ,
    enh039AndetAreal character varying(50) COLLATE pg_catalog."default" ,
    enh041LovligAnvendelse character varying(50) COLLATE pg_catalog."default" ,
    enh042DatoForTidsbegrænsetDispensation character varying(50) COLLATE pg_catalog."default" ,
    enh044DatoForDelvisIbrugtagningsTilladelse character varying(50) COLLATE pg_catalog."default" ,
    enh045Udlejningsforhold character varying(50) COLLATE pg_catalog."default" ,
    enh046OffentligStøtte character varying(50) COLLATE pg_catalog."default" ,
    enh047IndflytningDato character varying(50) COLLATE pg_catalog."default" ,
    enh048GodkendtTomBolig character varying(50) COLLATE pg_catalog."default" ,
    enh051Varmeinstallation character varying(50) COLLATE pg_catalog."default" ,
    enh052Opvarmningsmiddel character varying(50) COLLATE pg_catalog."default" ,
    enh053SupplerendeVarme character varying(50) COLLATE pg_catalog."default" ,
    enh060EnhedensAndelFællesAdgangsareal  integer,
    enh061ArealAfÅbenOverdækning integer,
    enh062ArealAfLukketOverdækningUdestue integer,
    enh063AntalVærelserTilErhverv integer,
    enh065AntalVandskylledeToiletter integer,
    enh066AntalBadeværelser integer,
    enh067Støjisolering character varying(50) COLLATE pg_catalog."default" ,
    enh101Gyldighedsdato character varying(50) COLLATE pg_catalog."default" ,
    enh127FysiskArealTilBeboelse integer,
    enh128FysiskArealTilErhverv integer,
    enh500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    enh068FlexboligTilladelsesart character varying(50) COLLATE pg_catalog."default" ,
    enh069FlexboligOphørsdato character varying(50) COLLATE pg_catalog."default" ,
    enh070ÅbenAltanTagterrasseAreal integer,
    enh102HerafAreal1 character varying(50) COLLATE pg_catalog."default" ,
    enh103HerafAreal2 character varying(50) COLLATE pg_catalog."default" ,
    enh104HerafAreal3 character varying(50) COLLATE pg_catalog."default" ,
    enh105SupplerendeAnvendelseskode1 character varying(50) COLLATE pg_catalog."default" ,
    enh106SupplerendeAnvendelseskode2 character varying(50) COLLATE pg_catalog."default" ,
    enh107SupplerendeAnvendelseskode3 character varying(50) COLLATE pg_catalog."default" ,
    enh071AdresseFunktion character varying(50) COLLATE pg_catalog."default" ,
    adresseIdentificerer character varying(50) COLLATE pg_catalog."default" ,
    etage character varying(50) COLLATE pg_catalog."default" ,
    opgang character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT EnhedList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.EnhedEjendomsrelationList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    ejerlejlighed character varying(50) COLLATE pg_catalog."default" ,
    enhed character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT EnhedEjendomsrelationList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.EtageList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    eta006BygningensEtagebetegnelse character varying(50) COLLATE pg_catalog."default" ,
    eta020SamletArealAfEtage integer,
    eta021ArealAfUdnyttetDelAfTagetage integer,
    eta022Kælderareal integer,
    eta023ArealAfLovligBeboelseIKælder integer,
    eta024EtagensAdgangsareal integer,
    eta025Etagetype character varying(50) COLLATE pg_catalog."default" ,
    eta026ErhvervIKælder character varying(50) COLLATE pg_catalog."default" ,
    eta500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT EtageList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.FordelingAfFordelingsarealList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    beboelsesArealFordeltTilEnhed integer,
    erhvervsArealFordeltTilEnhed integer,
    enhed character varying(50) COLLATE pg_catalog."default" ,
    fordelingsareal character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT FordelingAfFordelingsarealList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.FordelingsarealList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    for002Fordelingsarealnummer integer,
    for003ArealTilFordeling integer,
    for004FordelingsNøgle character varying(50) COLLATE pg_catalog."default" ,
    for500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    for005Navn character varying(50) COLLATE pg_catalog."default" ,
    for006Rest character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT FordelingsarealList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.GrundList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    gru009Vandforsyning character varying(50) COLLATE pg_catalog."default" ,
    gru010Afløbsforhold character varying(50) COLLATE pg_catalog."default" ,
    gru021Udledningstilladelse character varying(50) COLLATE pg_catalog."default" ,
    gru022MedlemskabAfSpildevandsforsyning character varying(50) COLLATE pg_catalog."default" ,
    gru023PåbudVedrSpildevandsafledning character varying(50) COLLATE pg_catalog."default" ,
    gru024FristVedrSpildevandsafledning character varying(50) COLLATE pg_catalog."default" ,
    gru025TilladelseTilUdtræden character varying(50) COLLATE pg_catalog."default" ,
    gru026DatoForTilladelseTilUdtræden character varying(50) COLLATE pg_catalog."default" ,
    gru027TilladelseTilAlternativBortskaffelseEllerAfledning character varying(50) COLLATE pg_catalog."default" ,
    gru028DatoForTilladelseTilAlternativBortskaffelseEllerAfledning character varying(50) COLLATE pg_catalog."default" ,
    gru029DispensationFritagelseIftKollektivVarmeforsyning character varying(50) COLLATE pg_catalog."default" ,
    gru030DatoForDispensationFritagelseIftKollektivVarmeforsyning character varying(50) COLLATE pg_catalog."default" ,
    gru500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    husnummer character varying(50) COLLATE pg_catalog."default" ,
    bestemtFastEjendom character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT GrundList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.GrundJordstykkeList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    grund character varying(50) COLLATE pg_catalog."default" ,
    jordstykke character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT GrundJordstykkeList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.OpgangList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    opg020Elevator character varying(50) COLLATE pg_catalog."default" ,
    opg500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    opg021HusnummerFunktion character varying(50) COLLATE pg_catalog."default" ,
    adgangFraHusnummer character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT OpgangList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.SagsniveauList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    niveautype character varying(50) COLLATE pg_catalog."default" ,
    sagstype character varying(50) COLLATE pg_catalog."default" ,
    sagsdataEtage character varying(50) COLLATE pg_catalog."default" ,
    stamdataEtage character varying(50) COLLATE pg_catalog."default" ,
    sagsdataEnhed character varying(50) COLLATE pg_catalog."default" ,
    stamdataEnhed character varying(50) COLLATE pg_catalog."default" ,
    stamdataBygning character varying(50) COLLATE pg_catalog."default" ,
    sagsdataBygning character varying(50) COLLATE pg_catalog."default" ,
    byggesag character varying(50) COLLATE pg_catalog."default" ,
    sagsdataTekniskAnlæg character varying(50) COLLATE pg_catalog."default" ,
    stamdataTekniskAnlæg character varying(50) COLLATE pg_catalog."default" ,
    stamdataOpgang character varying(50) COLLATE pg_catalog."default" ,
    sagsdataOpgang character varying(50) COLLATE pg_catalog."default" ,
    stamdataGrund character varying(50) COLLATE pg_catalog."default" ,
    sagsdataGrund character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT SagsniveauList_pkey PRIMARY KEY (id_lokalId)
);

CREATE TABLE IF NOT EXISTS public.TekniskAnlægList
(
    forretningshændelse character varying(50) COLLATE pg_catalog."default" ,
    forretningsområde character varying(50) COLLATE pg_catalog."default" ,
    forretningsproces character varying(50) COLLATE pg_catalog."default" ,
    id_namespace character varying(50) COLLATE pg_catalog."default" ,
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    registreringFra character varying(50) COLLATE pg_catalog."default" ,
    registreringsaktør character varying(50) COLLATE pg_catalog."default" ,
    registreringTil character varying(50) COLLATE pg_catalog."default" , 
    virkningFra character varying(50) COLLATE pg_catalog."default" ,
    virkningsaktør character varying(50) COLLATE pg_catalog."default" ,
    virkningTil character varying(50) COLLATE pg_catalog."default" ,
    status character varying(50) COLLATE pg_catalog."default" ,
    tek007Anlægsnummer character varying(50) COLLATE pg_catalog."default" ,
    tek020Klassifikation character varying(50) COLLATE pg_catalog."default" ,
    tek021FabrikatType character varying(50) COLLATE pg_catalog."default" ,
    tek022EksternDatabase character varying(50) COLLATE pg_catalog."default" ,
    tek023EksternNøgle character varying(50) COLLATE pg_catalog."default" ,
    tek024Etableringsår smallint,
    tek025TilOmbygningsår character varying(50) COLLATE pg_catalog."default" ,
    tek026StørrelsesklasseOlietank character varying(50) COLLATE pg_catalog."default" ,
    tek027Placering character varying(50) COLLATE pg_catalog."default" ,
    tek028SløjfningOlietank character varying(50) COLLATE pg_catalog."default" ,
    tek030Fabrikationsnummer character varying(50) COLLATE pg_catalog."default" ,
    tek031Typegodkendelsesnummer character varying(50) COLLATE pg_catalog."default" ,
    tek032Størrelse character varying(50) COLLATE pg_catalog."default" ,
    tek033Type character varying(50) COLLATE pg_catalog."default" ,
    tek034IndholdOlietank character varying(50) COLLATE pg_catalog."default" ,
    tek035SløjfningsfristOlietank character varying(50) COLLATE pg_catalog."default" ,
    tek036Rumfang character varying(50) COLLATE pg_catalog."default" ,
    tek037Areal integer,
    tek038Højde integer,
    tek039Effekt character varying(50) COLLATE pg_catalog."default" ,
    tek040Fredning character varying(50) COLLATE pg_catalog."default" ,
    tek042Revisionsdato character varying(50) COLLATE pg_catalog."default" ,
    tek045Koordinatsystem character varying(50) COLLATE pg_catalog."default" ,
    tek067Fabrikationsår smallint,
    tek068Materiale character varying(50) COLLATE pg_catalog."default" ,
    tek069SupplerendeIndvendigKorrosionsbeskyttelse character varying(50) COLLATE pg_catalog."default" ,
    tek070DatoForSenestUdførteSupplerendeIndvendigKorrosionsbeskyttelse character varying(50) COLLATE pg_catalog."default" ,
    tek072Sløjfningsår smallint,
    tek073Navhøjde integer,
    tek074Vindmøllenummer character varying(50) COLLATE pg_catalog."default" ,
    tek075Rotordiameter character varying(50) COLLATE pg_catalog."default" ,
    tek076KildeTilKoordinatsæt character varying(50) COLLATE pg_catalog."default" ,
    tek077KvalitetAfKoordinatsæt character varying(50) COLLATE pg_catalog."default" ,
    tek078SupplerendeOplysningOmKoordinatsæt character varying(50) COLLATE pg_catalog."default" ,
    tek101Gyldighedsdato character varying(50) COLLATE pg_catalog."default" ,
    tek102FabrikatVindmølle character varying(50) COLLATE pg_catalog."default" ,
    tek103FabrikatOliefyr character varying(50) COLLATE pg_catalog."default" ,
    tek104FabrikatSolcelleanlægSolvarme character varying(50) COLLATE pg_catalog."default" ,
    tek105OverdækningTank character varying(50) COLLATE pg_catalog."default" ,
    tek106InspektionsdatoTank character varying(50) COLLATE pg_catalog."default" ,
    tek107PlaceringPåSøterritorie character varying(50) COLLATE pg_catalog."default" ,
    tek109Koordinat character varying(50) COLLATE pg_catalog."default" ,
    tek500Notatlinjer character varying(50) COLLATE pg_catalog."default" ,
    tek110Driftstatus character varying(50) COLLATE pg_catalog."default" ,
    tek111DatoForSenesteInspektion character varying(50) COLLATE pg_catalog."default" ,
    tek112InspicerendeVirksomhed character varying(50) COLLATE pg_catalog."default" ,
    jordstykke character varying(50) COLLATE pg_catalog."default" ,
    husnummer character varying(50) COLLATE pg_catalog."default" ,
    enhed character varying(50) COLLATE pg_catalog."default" ,
    ejerlejlighed character varying(50) COLLATE pg_catalog."default" ,
    bygningPåFremmedGrund character varying(50) COLLATE pg_catalog."default" ,
    bygning character varying(50) COLLATE pg_catalog."default" ,
    grund character varying(50) COLLATE pg_catalog."default" ,
    CONSTRAINT TekniskAnlægList_pkey PRIMARY KEY (id_lokalId)
);
"""

In [13]:
run_sql(params,SQLcreateall)

## Insertion function to add items to the tables

The last step before we can start reading the BBR JSON file and adding items to the PostgresQL database is to create a function to insert new rows into the tables we just created.

In [4]:
def insert_bbr_from_dict(row_dict,table_name): # Takes a dictionary as input, corresponding to one table row, with keys corresponding to table columns and values equal to each parameter's value in this particular row, and inserts it into the PostgresQL table
    # We first convert the dictionary into a tuple (note that we could keep it as a dictionary if we used "row_factory=pg.rows.dict_row" in the connect function below)
    l=list()
    ks=row_dict.keys()
    for k in ks:
        l.append(row_dict[k])
    row_tuple=tuple(l)
    
    # Building the SQL query to insert values in the database.     
    sql ="INSERT INTO "+table_name+"("
    for k in ks:
        sql+=k+', '
    sql=sql[0:len(sql)-2]+') VALUES('
    for n in range(len(ks)):
        sql+='%s, '
    sql=sql[0:len(sql)-2]+') ON CONFLICT ON CONSTRAINT '+table_name+'_pkey DO UPDATE SET ('
    for k in ks:
        sql+=k+', '
    sql=sql[0:len(sql)-2]+') = ('
    for k in ks:
        sql+='EXCLUDED.'+k+', '
    sql=sql[0:len(sql)-2]+');'
    
    connector = None
    
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(params)
        # create a new cursor
        cur = connector.cursor()
        # execute the INSERT statement
        cur.execute(sql, row_tuple)
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
        
    except (Exception, pg.DatabaseError) as error:
        print(row_tuple)
        print(error)
        
    finally:
        if connector is not None:
            connector.close()

## Parsing BBR and adding items to the database

Now we can finally start parsing the BBR file and inserting values into our database. Because BBR is a very large dataset (if you download the whole thing), your computer will run out of memory when trying to parse it in one chunk. For this reason, we want to parse the file iteratively. We can do so using the ijson.parse function. It reads the file and records each "event" it encounters. An event can correspond to the start or end of an entry, a new parameter value being read, etc. Whenever an event is encountered, we can program the function to execute a particular action - such as adding a parameter to an incomplete row, or inserting a full row into the database. We also record how many events have been encountered, to be able to skip these events if we need to restart the function (the import will take a lot of time on a regular computer, with billions of events, and crashes happen).

In [5]:
def retrieve_values(objectlist,jsonfile,last_recorded_event=0):
    # objectlist is a list of the types of objects we want to record. May include BBRSagList, BygningList, BygningEjendomsrelationList, EjendomsrelationList, EnhedList, EtageList, FordelingAfFordelingsarealList, FordelingsarealList, GrundList, GrundJordstykkeList, OpgangList, SagsniveauList, TekniskAnlægList
    jsondata = open(jsonfile, encoding='utf8')
    parser=ijson.parse(jsondata)
    row_dict=dict()
    current_item_type=None # What type of item are we recording?
    current_parameter=None # What parameter are we recording?
    global current_event_number # Records the amount of events read, so that we can skip these if the function is restarted (using the last_recorded_event parameter) 
    current_event_number=0
    
    for prefix, event, value in parser: # Parse the json file, reading the name and value of each parameter for each building
        current_event_number+=1
        if current_event_number%100000==0: # This just tracks progress by regularly printing the event number
            print(current_event_number,current_item_type)
            clear_output(wait=True)

        if event == 'start_array': # We enter a new list of entries, so we are now reading a new type of item
            current_item_type=prefix

        elif  event == 'map_key' and current_item_type is not None: # We are about to read a new parameter, and the map_key value tells us the name of the parameter
            current_parameter=value
            
        elif event in ['string','number','null']: # We are reading the value of the parameter. Normally, "string", "number" and "null" should cover all values encountered in BBR
            row_dict[current_parameter]=value     
        
        elif event == 'end_map': # We start reading a new item, so we insert the previous row into the database and then reset the row 
             
            if current_event_number>last_recorded_event: # Only continue if we have not previously recorded this event
                if len(row_dict.values())>0 and current_item_type in objectlist:
                    insert_bbr_from_dict(row_dict,current_item_type) # The type of item is identical to the table name in the database
                
            row_dict=dict()

# Add these rows if you just want to test the function, so that it stops after a few seconds
        # if current_event_number==10000:
         #   break
        
# When restarting the function after an interruption, go a few 1000 events back. Otherwise there is a risk to record an incomplete building.


In [6]:
objectlist=['EjendomsrelationList']
# Only add to this list the types of objects you actually want to record.
# The options are 'BBRSagList' 'BygningList' 'BygningEjendomsrelationList' 'EjendomsrelationList' 'EnhedList' 'EnhedEjendomsrelationList' 'EtageList' 'FordelingAfFordelingsarealList' 'GrundList' 'GrundJordstykkeList' 'OpgangList' 'SagsniveauList' 'TekniskAnlægList'

In [9]:
retrieve_values(objectlist,jsonfile,1240830000)

2808600000 TekniskAnlægList


In [8]:
current_event_number

1240839198